# Time Series with Polars accelerated by NVIDIA RAPIDS
### by Matt Harrison, author of [Effective Polars](https://a.co/d/dMOeanT)

In [1]:
!nvidia-smi | head

Tue Feb  4 14:05:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off |   00000000:01:00.0 Off |                    0 |
| 30%   32C    P8             29W /  300W |      12MiB /  46068MiB |      0%      Default |


Install with UV

In [6]:
!pip install -q altair hvplot

In [8]:
!pip install -q --extra-index-url=https://pypi.nvidia.com  polars[gpu]

## Getting Data

In [10]:
!if [ ! -f "usa_stocks_30m.parquet" ]; then curl https://storage.googleapis.com/rapidsai/colab-data/usa_stocks_30m.parquet -o usa_stocks_30m.parquet; else echo "usa_stocks_30m.parquet found"; fi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  633M  100  633M    0     0  51.1M      0  0:00:12  0:00:12 --:--:-- 65.4M


In [11]:
import polars as pl
import polars.selectors as cs

pl.__version__

'1.14.0'

In [12]:
stocks_lazy = pl.scan_parquet("usa_stocks_30m.parquet")

In [13]:
stocks_lazy

## Check Types

In [14]:
(stocks_lazy
 .head(10)
 .collect(engine='gpu'))

__index_level_0__,datetime,open,high,low,close,volume,ticker
i64,datetime[ms],f64,f64,f64,f64,i64,str
0,1999-11-18 17:00:00,45.56,50.0,45.5,46.0,9275000,"""A"""
1,1999-11-18 17:30:00,46.0,47.69,45.82,46.57,3200900,"""A"""
2,1999-11-18 18:00:00,46.56,46.63,41.0,41.0,3830500,"""A"""
3,1999-11-18 18:30:00,41.0,43.38,40.37,42.38,3688600,"""A"""
4,1999-11-18 19:00:00,42.31,42.44,41.56,41.69,1584300,"""A"""
5,1999-11-18 19:30:00,41.69,42.19,41.62,41.88,808800,"""A"""
6,1999-11-18 20:00:00,41.94,42.75,41.87,42.19,1211500,"""A"""
7,1999-11-18 20:30:00,42.19,42.94,42.06,42.82,728600,"""A"""
8,1999-11-18 21:00:00,42.88,42.88,42.25,42.31,775900,"""A"""


In [15]:
(stocks_lazy
 .head(10)
 .collect(engine='gpu')
 .dtypes
 )

[Int64,
 Datetime(time_unit='ms', time_zone=None),
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 String]

## Time Aggregations

In [16]:
%%time
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by('year')
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 332 ms, sys: 175 ms, total: 507 ms
Wall time: 258 ms


year,__index_level_0__,open,high,low,close,volume
i32,f64,f64,f64,f64,f64,f64
2002,12971.554929,36.646697,36.783138,36.508293,36.646054,263078.324184
2018,53810.03182,113.188075,113.505029,112.859166,113.18192,273294.390298
2000,7353.462272,47.018504,47.262723,46.773424,47.019192,220800.759298
2024,69888.33292,208.398297,208.859837,207.91459,208.411046,256332.253163
2012,38487.307259,55.84671,55.982971,55.71153,55.849,391315.550438
…,…,…,…,…,…,…
2021,61896.670541,186.129599,186.63599,185.607674,186.126191,258173.930134
2007,25657.922046,54.014047,54.17905,53.846799,54.013655,409269.848016
2008,27858.507801,44.856341,45.101187,44.606632,44.852959,591277.367665


In [17]:
%%time
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by('year')
 .agg(cs.numeric().mean())
 .collect() # run on CPU
 )


CPU times: user 5.68 s, sys: 1.72 s, total: 7.4 s
Wall time: 333 ms


year,__index_level_0__,open,high,low,close,volume
i32,f64,f64,f64,f64,f64,f64
2012,38487.307259,55.84671,55.982971,55.71153,55.849,391315.550438
2004,18736.99736,42.591014,42.691719,42.491388,42.593011,284706.816203
2006,23862.215936,49.509078,49.636308,49.383189,49.510772,344997.896297
1998,1566.031221,48.049922,48.205337,47.894569,48.052328,127224.481768
2005,21309.285447,46.071606,46.182677,45.960145,46.071621,327573.030487
…,…,…,…,…,…,…
2017,51493.026746,100.52437,100.727694,100.321566,100.525947,243621.500928
2002,12971.554929,36.646697,36.783138,36.508293,36.646054,263078.324184
2009,30924.07596,35.874852,36.027689,35.724714,35.877364,693840.277734


In [18]:
%%time
# groupby two items
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by(['ticker','year'])
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 283 ms, sys: 160 ms, total: 443 ms
Wall time: 241 ms


ticker,year,__index_level_0__,open,high,low,close,volume
str,i32,f64,f64,f64,f64,f64,f64
"""EIX""",2003,17876.5,16.478191,16.526376,16.430221,16.480795,94742.125921
"""ITW""",2009,37466.0,38.656422,38.79906,38.51955,38.660826,280829.59198
"""PM""",2019,36748.5,81.164758,81.35803,80.974375,81.166366,289072.751838
"""CPT""",2018,66601.0,88.849166,89.035491,88.656787,88.846904,29964.65097
"""BK""",2000,8161.5,46.533491,46.737838,46.32557,46.531653,149850.153092
…,…,…,…,…,…,…,…
"""MCD""",2005,24421.0,31.900816,31.978499,31.825008,31.902018,440495.628248
"""CAG""",2011,44000.5,24.414817,24.465691,24.364893,24.416073,291631.645477
"""DVN""",2011,21960.5,75.831152,76.082619,75.571822,75.82721,246032.207824


In [19]:
%%time
# sort results
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year())
 .group_by(['ticker','year'])
 .agg(cs.numeric().mean())
 .sort(['ticker', 'year'])
 .collect(engine='gpu')
 )

CPU times: user 322 ms, sys: 141 ms, total: 463 ms
Wall time: 248 ms


ticker,year,__index_level_0__,open,high,low,close,volume
str,i32,f64,f64,f64,f64,f64,f64
"""A""",1999,189.0,47.51504,47.881187,47.208522,47.554354,275892.612137
"""A""",2000,2011.5,69.742293,70.320141,69.139541,69.718236,222762.461727
"""A""",2001,5247.0,33.358924,33.59634,33.139426,33.366602,170170.826833
"""A""",2002,8477.5,22.984785,23.118953,22.851465,22.981314,189420.178133
"""A""",2003,11733.5,20.174097,20.250863,20.0986,20.177482,156302.885135
…,…,…,…,…,…,…,…
"""ZTS""",2020,24192.5,144.723845,145.203939,144.254515,144.727793,92883.735671
"""ZTS""",2021,27528.5,189.112176,189.522102,188.718892,189.122105,76024.96138
"""ZTS""",2022,30978.0,171.467585,171.940174,170.958298,171.446213,115138.105218


In [20]:
%%time
# let's try using .group_by_dynamic
(stocks_lazy
 .set_sorted('datetime')
 .group_by_dynamic(index_column='datetime', every='1y', group_by='ticker')
 .agg(cs.numeric().mean())
 .collect(engine='gpu')
 )

CPU times: user 11.8 s, sys: 1.71 s, total: 13.5 s
Wall time: 712 ms


ticker,datetime,__index_level_0__,open,high,low,close,volume
str,datetime[ms],f64,f64,f64,f64,f64,f64
"""A""",1999-01-01 00:00:00,189.0,47.51504,47.881187,47.208522,47.554354,275892.612137
"""A""",2000-01-01 00:00:00,2011.5,69.742293,70.320141,69.139541,69.718236,222762.461727
"""A""",2001-01-01 00:00:00,5247.0,33.358924,33.59634,33.139426,33.366602,170170.826833
"""A""",2002-01-01 00:00:00,8477.5,22.984785,23.118953,22.851465,22.981314,189420.178133
"""A""",2003-01-01 00:00:00,11733.5,20.174097,20.250863,20.0986,20.177482,156302.885135
…,…,…,…,…,…,…,…
"""ZTS""",2020-01-01 00:00:00,24192.5,144.723845,145.203939,144.254515,144.727793,92883.735671
"""ZTS""",2021-01-01 00:00:00,27528.5,189.112176,189.522102,188.718892,189.122105,76024.96138
"""ZTS""",2022-01-01 00:00:00,30978.0,171.467585,171.940174,170.958298,171.446213,115138.105218


In [21]:
%%time
# what's going on?
(stocks_lazy
 .set_sorted('datetime')
 .group_by_dynamic(index_column='datetime', every='1y', group_by='ticker')
 .agg(cs.numeric().mean())
 .collect(engine=pl.GPUEngine(device=0, raise_on_fail=True))
 )

ComputeError: 'cuda' conversion failed: NotImplementedError: ('Query execution with GPU not possible: unsupported operations.\nThe errors were:\n- NotImplementedError: dynamic group by', [NotImplementedError('dynamic group by')])

## Comparing Two Stocks

In [22]:
%%time
# monthly results
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .sort(['ticker', 'year', 'month'])
 .collect(engine='gpu')
 )

CPU times: user 307 ms, sys: 135 ms, total: 441 ms
Wall time: 228 ms


ticker,year,month,__index_level_0__,open,high,low,close,volume
str,i32,i8,i64,f64,f64,f64,f64,i64
"""A""",1999,11,97,46.56,50.0,45.82,46.57,9275000
"""A""",1999,12,378,79.5,80.0,78.0,78.94,1393300
"""A""",2000,1,638,78.75,78.94,73.0,73.75,549700
"""A""",2000,2,898,114.0,115.44,111.25,114.25,730500
"""A""",2000,3,1197,156.0,161.0,149.56,158.0,1179900
…,…,…,…,…,…,…,…,…
"""ZTS""",2023,11,35938,181.09,181.39,180.91,181.07,1393038
"""ZTS""",2023,12,36219,201.13,201.92,200.65,201.06,1470827
"""ZTS""",2024,1,36512,199.29,199.83,198.89,199.37,850382


In [23]:
%%time
# pivot so columns are close values
(stocks_lazy
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .with_columns(date=pl.date('year', 'month', day=1))
 .collect(engine='gpu')
 .pivot(index='date', values='close', on='ticker')
 .sort('date')
 )

CPU times: user 12.8 s, sys: 2.66 s, total: 15.4 s
Wall time: 968 ms


date,APA,CMA,REG,SRE,DHR,MAR,CL,VRSN,MNST,GEN,ROK,URI,CEG,O,UHS,VTR,LYV,DLR,MAA,AXP,CLX,MRO,K,ES,BG,PAYC,CDW,J,PAYX,WRB,PH,RSG,TSLA,AKAM,AEP,UPS,…,AMP,SPLK,OTIS,TMUS,TEAM,BBWI,AFL,CRWD,FOX,CFG,AAPL,WMB,TDG,DDOG,ABNB,DD,KHC,PANW,ABBV,MPWR,DASH,QRVO,GEHC,ALLE,WBD,PYPL,ETSY,UBER,DOW,TTD,FTV,VTRS,CCEP,BALL,EG,RVTY,CTRA
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1998-01-01,36.31,94.63,27.81,null,64.88,71.0,74.81,25.63,0.9,null,55.94,23.88,null,26.94,50.44,null,null,null,29.88,89.62,79.44,34.19,50.06,null,null,null,null,null,51.0,null,48.88,null,null,null,51.63,null,…,null,null,null,null,null,null,54.44,null,null,null,0.17,29.31,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-02-01,35.5,100.75,27.63,null,73.38,75.75,81.63,33.87,1.12,null,60.51,27.38,null,26.88,52.19,null,null,null,29.19,90.44,87.94,34.63,46.44,null,null,null,null,null,52.51,null,46.75,null,null,null,50.19,null,…,null,null,null,null,null,null,62.13,null,null,null,0.21,34.69,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-03-01,38.63,107.88,26.5,null,76.56,82.5,88.37,46.25,1.28,null,61.25,26.44,null,27.19,58.5,null,null,null,28.38,97.81,89.88,40.31,44.69,null,null,null,null,null,59.38,null,52.56,null,null,null,51.06,null,…,null,null,null,null,null,null,66.56,null,null,null,0.24,33.93,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-04-01,38.06,104.63,26.25,null,78.0,38.38,91.25,48.0,1.75,null,57.44,35.82,null,27.25,59.75,null,null,null,29.06,108.44,90.13,38.81,43.5,null,null,null,null,null,58.88,null,52.31,null,null,null,50.63,null,…,null,null,null,null,null,null,67.19,null,null,null,0.25,35.69,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1998-05-01,36.88,69.38,26.63,null,76.5,35.44,90.69,41.13,2.09,null,58.88,37.25,null,26.69,58.25,28.63,null,null,27.94,106.06,84.75,37.19,41.94,null,null,null,null,null,56.88,null,45.25,null,null,null,47.63,null,…,null,null,null,null,null,null,68.25,null,null,null,0.27,33.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-11-01,40.72,45.73,63.73,74.33,225.0,210.62,78.79,215.39,57.03,22.37,275.78,486.44,126.56,54.48,139.06,46.1,90.93,138.88,126.29,170.76,144.14,28.04,53.48,60.7,109.9,184.14,219.51,137.88,122.08,72.56,435.3,161.84,252.12,115.97,81.04,153.26,…,353.51,151.96,85.8,150.46,199.22,33.29,83.35,237.07,29.68,27.77,192.01,36.96,1010.0,118.24,131.2,72.01,35.32,null,143.29,558.81,null,96.91,73.73,107.35,11.84,59.52,78.49,56.94,51.92,79.79,69.0,9.48,null,55.4,416.42,90.58,28.18
2023-12-01,37.7,57.21,68.19,77.29,234.04,225.95,79.74,220.37,57.71,23.4,312.56,584.64,121.19,58.58,155.16,50.96,95.04,139.24,139.12,188.24,146.24,25.9,55.94,64.36,110.34,209.02,229.26,130.8,129.64,74.05,463.15,167.11,262.84,120.47,84.58,163.6,…,382.3,152.5,90.96,160.98,247.55,43.72,84.53,260.97,28.36,34.44,199.2,37.4,1015.0,124.4,149.77,77.18,37.74,null,155.37,645.76,null,114.88,78.27,128.42,12.61,63.79,89.37,63.41,55.84,78.09,74.0,10.87,null,60.55,411.96,111.21,26.53
2024-01-01,36.82,57.0,67.73,76.97,245.01,243.85,84.58,207.78,59.13,24.29,309.1,653.45,123.65,59.74,160.87,50.58,93.97,145.97,137.15,204.82,146.9,24.98,58.03,64.58,102.1,207.62,231.39,138.52,123.73,83.58,479.0,172.92,250.48,125.36,84.52,161.29,…,397.66,153.72,91.99,165.84,258.51,45.74,85.9,307.45,30.

In [24]:
%%time
# plot AMZN vs AAPL
(stocks_lazy
 .filter((pl.col('ticker') == 'AMZN') | (pl.col('ticker') == 'AAPL'))
 .with_columns(year=pl.col('datetime').dt.year(),
               month=pl.col('datetime').dt.month())
 .group_by(['ticker','year', 'month'])
 .agg(cs.numeric().max())
 .sort(['ticker', 'year', 'month'])
 .collect(engine='gpu')
 .with_columns(date=pl.date('year', 'month', day=1))
 # pivot not needed for altair
 # .pivot(index='date', values='close', on='ticker')
 .plot.line(x='date', y='close', color='ticker')
 )

CPU times: user 394 ms, sys: 126 ms, total: 520 ms
Wall time: 324 ms


alt.Chart(...)

## Conclusion

- Easy to install
- Works with existing Polars code
- Up to 13x speed improvement

Connect with me on social media and check out *Effective Polars* by Matt Harrison for getting started with the library.


# END